# OpenAI Functions

This notebook goes over how run an agent that uses OpenAI Functions

In [5]:
# !pip install openai agents_deconstructed

In [14]:
from langchain.agents import tool
from langchain.schema.agent import AgentFinish
from langchain.callbacks.manager import (
    trace_as_chain_group,
)
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import ChatOpenAI
from langchain.tools.convert_to_openai import format_tool_to_openai_function

In [4]:
from agents_deconstructed import openai_functions

In [5]:
# Create a dummy tool
@tool
def run_code(query: str):
    """Input should be Python code. Use this to do math."""
    return eval(query)

In [7]:
functions = [dict(format_tool_to_openai_function(t)) for t in [run_code]]

In [15]:
# Create our 'agent' - this is a runnable
agent = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{question}"),
    MessagesPlaceholder(variable_name="intermediate_steps"),
]) | ChatOpenAI(temperature=0).bind(functions=functions) | openai_functions.FunctionsAgentOutputParser()

In [16]:
# Use this to group all traces together
with trace_as_chain_group("agent-run") as group_manager:
    # Keep track of steps taken
    steps = []
    # Get the first response from the agent
    action = agent.invoke({
        "question": "whats 2 + 2",
        "intermediate_steps": openai_functions.format_intermediate_steps(steps)
    }, config={"callbacks": group_manager})

    # While it's not finished, iterate
    while not isinstance(action, AgentFinish):
        # Select the tool to use
        tool = {
            "run_code": run_code,
        }[action.tool]
        # Run the tool
        observation = tool.run(action.tool_input, callbacks=group_manager)
        # Construct intermediate steps
        steps += [(action, observation)]
        # Log the steps
        print(steps)
        # Do the next iteration
        action = agent.invoke({
            "question": "whats 2 + 2",
            "intermediate_steps": openai_functions.format_intermediate_steps(steps)
        }, config={"callbacks": group_manager})

[(FunctionsAgentAction(tool='run_code', tool_input={'query': '2 + 2'}, log="\nInvoking: `run_code` with `{'query': '2 + 2'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'run_code', 'arguments': '{\n  "query": "2 + 2"\n}'}}, example=False)]), 4)]


In [17]:
print(action)

AgentFinish(return_values={'output': '2 + 2 equals 4.'}, log='2 + 2 equals 4.')
